# Análise Marketplace

Esse problema de marketplace foi me passado por um amigo, não tenho certeza da origem, provavelmente recebeu em um processo seletivo. Imagino que esse problema não seja mais usado para esse fim, então eu vou divulgá-lo com a minha análise como portifólio.

-----

# **Descrição do problema**


Dentro da empresa existe uma equipe que é responsável por definir as características de produtos
que vão ser construídos. Para uma categoria de liquidificadores, por exemplo, essa equipe fica
responsável por definir a cor, volume do copo, material do copo ou mesmo se vai ter um filtro para
suco de frutas.


Para definir as características das diferentes categorias de produtos, a equipe procura entender
quando o cliente se interessa por um produto. É nesse ponto que o time do Labs é chamado. O seu
objetivo principal é desenvolver um modelo que consiga classificar o produto entre "interessante" e
"não interessante" dados os atributos do mesmo. Além disso, é desejável que você também consiga
indicar quais são os atributos e respectivos valores que mais impulsionam o interesse do cliente.
Mais importante que o resultado final é a sua estratégia de solução. Deixe claro cada passo da sua
solução e explique as decisões tomadas durante o processo. Também é importante que você nos
mande o seu código desenvolvido (Python ou Scala). Uma sugestão é usar Jupyter Notebook.
Assim, você consegue explicar cada ponto da sua estratégia de forma mais explícita.
Esperamos receber de você um documento com a explicação do trabalho desenvolvido.


 **Bom trabalho!**


Conjunto de dados


O conjunto de dados possui os atributos de cada produto de uma categoria de panelas e se houve ou
não interesse do cliente ao visualizar o produto no site. Cada linha é o registro da visualização de
um cliente em determinado momento.

-----

Importando as bibliotecas e o dataframe

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('problem1_dataset.csv')

**Análise exploratória dos dados**

In [ ]:
df.sample(10)

Reutilização de uma função útil obtina no kaggle

In [ ]:
def resume(df):
    """
    Objective: For a given dataframe this function provides information
    regarding Missing and Unique values per column.

    Input: param df: Dataframe to check the information.

    Output: return summary: a dataframe with columns providing summary per column of the input dataframe.
    
    """
    df = df.copy()
    #print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values
    summary['Missing Percentage'] = df.isnull().sum().values/len(df)
    summary['Uniques'] = df.nunique().values
    return summary

In [ ]:
resume(df)

In [ ]:
df.describe()

Importando bibliotecas de visualização de dados

In [ ]:
import matplotlib.pyplot as plt
import matplotlib_inline
import seaborn as sns

In [ ]:
sns.histplot(df['INTERESTED'])

Os dados retulados estão um pouco desbalanceados

In [ ]:
sns.scatterplot(x=df['ITEM_ID'],y=df['INTERESTED'])

In [ ]:
sns.boxplot(df['ITEM_ID'])

In [ ]:
df['SESSION_ID'].max()

In [ ]:
df1 = df[df['SESSION_ID']>1000]
df2 = df[df['SESSION_ID']>1000]

In [ ]:
plot = sns.histplot(df1['ITEM_ID'],bins=10)

In [ ]:
plot2 = sns.histplot(df2['ITEM_ID'],bins=10)

O atributo SESSION_ID está relacionado com o comportamento de cada usuário no sistema, para esse problema, não estamos interessados em classificar usuários, e esse atributo deve ser removido para evitar vazamento de informação.

**Ação 1- Remover coluna SESSION_ID**

In [ ]:
sns.boxplot(df['ITEM_ID'])

O mesmo efeito acontece com o atributo ITEM_ID, estamos focados nas características de um produto que são interessantes e não nos produtos que possuem essas características.

**Ação 2- Remover coluna ITEM_ID**

In [ ]:
sns.histplot(df['ALTURA'],kde=True)

In [ ]:
sns.histplot(df['CAPACIDADE_(L)'],kde=True)

In [ ]:
sns.histplot(df['LARGURA'],kde=True)

In [ ]:
grafico = sns.histplot(data=df, x='PESO', kde=True)
grafico.set(ylim=1000)

In [ ]:
sns.histplot(df['PROFUNDIDADE'], kde=True)

In [ ]:
sns.histplot(df['TEMPO_GARANTIA'], kde=True)

In [ ]:
df['TEMPO_GARANTIA'].unique()

Os valores de TEMPO_GARANTIA está com unidades misturadas, meses e dias

**Ação 3 - converter valores de TEMPO_GARANTIA para meses**

In [ ]:
df['COR'].unique()

In [ ]:
df['FORMATO'].unique()

In [ ]:
df['COMPOSICAO'].unique()

In [ ]:
df['MARCA'].unique()

In [ ]:
df['TIPO_PRODUTO'].unique()

In [ ]:
df[df['TIPO_PRODUTO']=='WOK'].TIPO_WOK.value_counts()

In [ ]:
df[df['TIPO_PRODUTO']!='WOK'].TIPO_WOK.value_counts()

A panela do tipo wok é uma panela com o fundo redondo como a da imagem abaixo.

![Alt text](wok.jpeg)

A informação contida no atributo TIPO_PRODUTO quando possue o valor WOK está todo contido no outro atributo TIPO_WOK. Sendo assim o valor WOK pode ser modificado para o valor neutro COMUM 

**Ação 4 - Substituir valor WOK em TIPO_PRODUTO por valor COMUM**

In [ ]:
df[df['TIPO_PRODUTO']=='TAMPA'].TEM_TAMPA.value_counts()

In [ ]:
df[df['TIPO_PRODUTO']!='TAMPA'].TEM_TAMPA.value_counts()

A informação contida no atributo TIPO_PRODUTO quando possue o valor TAMPA está todo contido no outro atributo TEM_TAMPA. Sendo assim o valor TAMPA pode ser modificado para o valor neutro COMUM 

**Ação 5 - Substituir valor TAMPA em TIPO_PRODUTO por valor COMUM**

In [ ]:
df[df['TIPO_PRODUTO']!='COZI-PASTA'].MARCA.value_counts()

In [ ]:
df[df['TIPO_PRODUTO']!='ESPAGUETEIRA'].MARCA.value_counts()

Uma panela TIPO_PRODUTO que é ESPAGUETEIRA provavelvente é uma panela que tem ou um escorredor de espague interno (tipo 1) ou uma tampa com um escorredor (tipo 2) como nas figuras abaixo.

![Alt text](espagueteira_1.webp)  ![Alt text](espagueteira_2.webp)

Já o tipo COZI-PASTA é o mesmo que uma espagueteira tipo 1.

Existe uma sobreposição de informação devido a uma ação de marketing. Talvez o impacto dessa ação possa ser medida pelo modelo.

**Tratamento de dados faltantes**

In [ ]:
resumo = resume(df)

In [ ]:
resumo =resumo[resumo['Missing Percentage']!=0]
resumo =resumo[resumo['Missing Percentage']<0.1]
resumo = resumo.drop(18,axis=0)#Atributo já tratado
resumo

**AÇÃO 6 - As linhas serão removidas para os atributos que têm menos de 10% dos dados faltantes**

Para os atributos categóricos 

**AÇÃO 7 - COMPOSIÇÃO, FORMATO e TIPO_PRODUTO**  possuem **dados faltantes** e **serão substituidos por**


**sem_composição, sem_formato e sem_tipo** respectivamente.

In [ ]:
resumo2 = resume(df)
resumo2 = resumo2[resumo2['dtypes']== 'float64']
resumo2 = resumo2[resumo2['Missing Percentage']!= 0]
resumo2 = resumo2.drop([1,6,11,19,18],axis=0)#Dados faltantes já tratados
resumo2

**Ação 8 - Os valores faltantes nas variáveis numéricas CAPACIDADE_(L), PESO e TEMPO_GARANTIA terão os dados faltantes subtituidos por 0**



In [ ]:
resumo3 = resume(df)
resumo3 = resumo3[resumo3['Uniques']<=3]
resumo3

Já que a porcentagem de dados faltantes é baixa e não afetaria a quantidade significativa de informação é possível removẽ-las.

**Ação 9 - Para os atributos categóricos binários remover as linhas dos dados faltantes**

In [ ]:
def feat_eng(dataframe):
    df=dataframe.copy()

    df.drop('SESSION_ID',axis=1,inplace=True)#Ação 1

    df.drop('ITEM_ID',axis=1,inplace=True)#Ação 2

    df['TEMPO_GARANTIA'].replace(1.0,12.0,inplace=True)#Ação 3
    df['TEMPO_GARANTIA'].replace(300,10.0,inplace=True)
    df['TEMPO_GARANTIA'].replace(60,3.0,inplace=True)

    df['TIPO_PRODUTO'].replace('WOK','COMUM',inplace=True)#Ação 4

    df['TIPO_PRODUTO'].replace('TAMPA','COMUM',inplace=True)#Ação 5

    #df.dropna(subset=['ALTURA','COR','LARGURA','MARCA','PROFUNDIDADE','ITEM_PRICE'],axis=0,inplace=True)#Ação 6
    #a procentagem somada das colunas removidas chegava a 40% retirando informações importantes de outroa atributos
    df['ALTURA'].fillna(0.0,inplace=True)
    df['COR'].fillna('Sem_cor',inplace=True)
    df['LARGURA'].fillna(0.0,inplace=True)
    df['MARCA'].fillna('sem_marca',inplace=True)
    df['PROFUNDIDADE'].fillna(0.0,inplace=True)
    df['ITEM_PRICE'].fillna(0.0,inplace=True)
    
    
    df['COMPOSICAO'].fillna('sem_composição',inplace=True)#Ação 7
    df['FORMATO'].fillna('sem_formato',inplace=True)
    df['TIPO_PRODUTO'].fillna('sem_tipo',inplace=True)

    df['CAPACIDADE_(L)'].fillna(0,inplace=True)#Ação 8
    df['PESO'].fillna(0,inplace=True)
    df['TEMPO_GARANTIA'].fillna(0,inplace=True)

    #df.dropna(subset=['PARA_LAVA_LOUCAS','PARA_MICRO_ONDAS'],axis=0, inplace=True)#Ação 9
    df['PARA_LAVA_LOUCAS'].fillna(0.0,inplace=True)
    df['PARA_MICRO_ONDAS'].fillna(0.0,inplace=True)

    return df

    
    

In [ ]:
def num_encode(dataf):
    binary_feat = ['TEM_FERRO_FUNDIDO','TEM_GRELHA','TIPO_WOK','PARA_LAVA_LOUCAS','PARA_MICRO_ONDAS']
    #replecement = [['NAO',0.0],['No',0.0],['no',0.0],['SIM',1.0],['Yes',1.0],['yes',1.0]]
    for binary in binary_feat:
        dataf[binary].replace('NAO',0.0,inplace=True)
        dataf[binary].replace('No',0.0,inplace=True)
        dataf[binary].replace('no',0.0,inplace=True)
        dataf[binary].replace('SIM',1.0,inplace=True)
        dataf[binary].replace('Yes',1.0,inplace=True)
        dataf[binary].replace('yes',1.0,inplace=True)
    
    return dataf


In [ ]:
df

In [ ]:
data = feat_eng(df)
data

In [ ]:
data = num_encode(data)
data

In [ ]:
sns.histplot(data['INTERESTED'])

Importando bibliotecas de processamento e estimadores

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from xgboost import XGBClassifier

In [ ]:
y = data['INTERESTED']
X = data.drop(['INTERESTED'],axis=1)

In [ ]:
def one_hot_enc(data_x):
    categ_cols=['COMPOSICAO','COR','FORMATO','MARCA','TIPO_PRODUTO']
    ohenc = OneHotEncoder(handle_unknown='ignore', sparse=False)
    ohenc_cols = pd.DataFrame(ohenc.fit_transform(data_x[categ_cols]))
    #ohenc_cols.categories_[0]
    ohenc_cols.index = data_x.index
    data_x = data_x.drop(categ_cols, axis=1)
    return pd.concat([data_x,ohenc_cols],axis=1)

In [ ]:
data = one_hot_enc(data)
data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data,y)

In [ ]:
model = XGBClassifier()

In [ ]:
X_train

In [ ]:
model.fit(X_train,y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test,predictions)
accuracy

A acurácia desse modelo é de 92%, o que é um resultado muito bom.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cm = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

Mas se observarmos a matriz de confusão é possível perceber que o modelo tem acertado muito os casos não interessantes e errado a previsão de situações interessantes. Isso porque as classes alvo estão desbalanceadas.

In [ ]:
from imblearn.combine import SMOTETomek

In [ ]:
smt = SMOTETomek()
X_trn_smt,y_trn_smt = smt.fit_sample(data, y)

In [ ]:
sns.histplot(y_trn_smt)

In [ ]:
model2 = XGBClassifier()
model2.fit(X_trn_smt,y_trn_smt)

In [ ]:
predictions2 = model2.predict(X_test)

In [ ]:
accu2 = accuracy_score(predictions2,y_test)
accu2

A acurácia do modelo melhorado caiu para 72%.

In [ ]:
cm2 = confusion_matrix(y_test, predictions2)
disp2 = ConfusionMatrixDisplay(confusion_matrix=cm2)
disp2.plot()

Entretanto, se olharmos para a matriz de confusão, vemos que agora os resultados estão balanceados e podemos confiar melhor neles.

In [ ]:
from sklearn.inspection import permutation_importance
r = permutation_importance(model2, X_train, y_train,n_repeats=30,random_state=0)

In [ ]:
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{data.iloc[i]:<8}"
            f"{r.importances_mean[i]:.3f}"
                f" +/- {r.importances_std[i]:.3f}")

In [ ]:
r.importances

In [ ]:
importance_df = pd.DataFrame({"gain":model2.feature_importances_}, index=data.columns).sort_values("gain", ascending=False)

In [ ]:
importance_df.head(10)